# Text simplification 
using code provided by https://github.com/chaojiang06/wiki-auto
Paper link: https://arxiv.org/abs/2005.02324

You need tensorflow 2 to be installed to run the code below!

## Data loading

## Data Preprocessing

In [27]:
#preprocess test_data/raw test_data/tokenized test_data/binarized
def preprocess_data(raw_data_dir,tokenized_data_dir,binarized_data_dir):
    !rm -r $binarized_data_dir/*
    !rm -r $tokenized_data_dir/*

    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/test.src --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/test.tok.src
    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/test.dst --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/test.tok.dst

    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/valid.src --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/valid.tok.src
    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/valid.dst --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/valid.tok.dst


    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/train.src --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/train.tok.src
    !python simplification/preprocess/anonymize_wordpiece.py --input $raw_data_dir/train.dst --vocab simplification/preprocess/vocab.txt --output  $tokenized_data_dir/train.tok.dst


    # Creates binarized fairseq dataset
    !python simplification/preprocess.py --workers 5 --source-lang src --target-lang dst --trainpref $tokenized_data_dir/train.tok --validpref $tokenized_data_dir/valid.tok --testpref $tokenized_data_dir/test.tok --destdir  $binarized_data_dir --padding-factor 1 --joined-dictionary --srcdict simplification/preprocess/vocab_count.txt


In [28]:
preprocess_data('simplification/test_data/raw','simplification/test_data/tokenized','simplification/test_data/binarized')

Namespace(alignfile=None, cpu=False, criterion='cross_entropy', dataset_impl='cached', destdir='simplification/test_data/binarized', fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=1, seed=1, source_lang='src', srcdict='simplification/preprocess/vocab_count.txt', target_lang='dst', task='translation', tbmf_wrapper=False, tensorboard_logdir='', testpref='simplification/test_data/tokenized/test.tok', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, trainpref='simplification/test_data/tokenized/train.tok', user_dir=None, validpref='simplification/test_data/tokenized/valid.tok', workers=5)
| [src] Dictionary: 30525 types
| [src] simplification/test_data/tokenized/train.tok.src: 50 sents, 2205 tokens,

## Text simplification generation

In [58]:
def generate_simplified_text(binarized_data_dir,output_file,checkpoint_file, gpu_id="", split="test"):
    print("Generating using checkpoint" , {checkpoint_file})
    print()
    !echo $output_file'.aner'

    !export CUDA_VISIBLE_DEVICES=$gpu_id 
    !python simplification/generate.py $binarized_data_dir --path $checkpoint_file --batch-size 32  --beam 1 --nbest 1 --user-dir simplification/my_model/ --print-alignment --gen-subset $split > $output_file'.aner'

    !python simplification/postprocess/bpe.py  --out_anon $output_file'.aner' --denon $output_file --ignore_lines 5 --wp 1

    !rm $output_file'.aner'
generate_simplified_text('simplification/test_data/binarized','simplification/0000','simplification/checkpoints/checkpoint_wiki_auto.pt',0,"test")

Generating using checkpoint {'simplification/checkpoints/checkpoint_wiki_auto.pt'}

simplification/0000.aner


In [50]:
generate_simplified_text('simplification/test_data/binarized','simplification/0000.txt','simplification/checkpoints/checkpoint_wiki_auto.pt',0,"test")

Generating using checkpoint {'simplification/checkpoints/checkpoint_wiki_auto.pt'}

usage: generate.py [-h] [--no-progress-bar] [--log-interval N]
                   [--log-format {json,none,simple,tqdm}]
                   [--tensorboard-logdir DIR] [--tbmf-wrapper] [--seed N]
                   [--cpu] [--fp16] [--memory-efficient-fp16]
                   [--fp16-init-scale FP16_INIT_SCALE]
                   [--fp16-scale-window FP16_SCALE_WINDOW]
                   [--fp16-scale-tolerance FP16_SCALE_TOLERANCE]
                   [--min-loss-scale D]
                   [--threshold-loss-scale THRESHOLD_LOSS_SCALE]
                   [--user-dir USER_DIR]
                   [--criterion {adaptive_loss,binary_cross_entropy,composite_loss,cross_entropy,label_smoothed_cross_entropy,masked_lm_loss}]
                   [--optimizer {adadelta,adafactor,adagrad,adam,lamb,nag,sgd}]
                   [--lr-scheduler {cosine,fixed,inverse_sqrt,polynomial_decay,reduce_lr_on_plateau,triangular}